# Epeak_Eiso_cosmology.ipynb
We have searched for a $E_\text{peak}- E_\text{iso}$ fit in the [Epeak-Eiso_corelation.ipynb](https://github.com/joanalnu/oab-inaf/blob/main/Epeak_Eiso_correlation.ipynb) and computed $E_\text{iso}$ from respective fluences $S$ in dependency of cosmology in [Eiso_from_fluence.ipynb](https://github.com/joanalnu/oab-inaf/blob7main/Eiso_from_fluence.ipynb). In this notebook, we employ all that to constrain the cosmological parameters, searching for $E_\text{peak}-E_\text{iso}$ fit with less scatter (i.e. shorter point-fit distances), indicating the 'best cosmology'.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.integrate import cumulative_trapezoid
from scipy.stats import norm

In [ ]:
# read data
df = pd.read_csv('table.csv')

Epeak = np.log10(df['Epeak'])
Epeak_err = np.log10(df['Epeak_err'])
z = df['z']
S = df['Fluence']  # ergs/cm2
S_err = df['Fluence_err']


# Convert H0 to cgs units: H0 [s^-1] = H0 [km/s/Mpc] * (km/Mpc to 1/s conversion)
# 1 Mpc = 3.086e24 cm, so H0_cgs = H0_kmsMpc * 1e5 / 3.086e24
H0_cgs = 70.0 * 1e5 / 3.086e24  # s^-1
c = 29979245800  # cm/s
Om_default = 0.3
Ode_default = 0.7
c = 2.99792458e10  # cm/s

def luminosity_distance(redshift, Ho=H0_cgs, O_m=Om_default, O_DE=Ode_default):
    """
    Calculate luminosity distance for given redshift(s) and cosmological parameters.

    Parameters:
    -----------
    redshift : float or array
        Redshift value(s)
    Ho : float
        Hubble constant in cgs units (s^-1)
    O_m : float
        Matter density parameter
    O_DE : float
        Dark energy density parameter

    Returns:
    --------
    float or array
        Luminosity distance in cm
    """

    def luminosity_distance_single(z, Ho=Ho, O_m=O_m, O_DE=O_DE):
        def integrand(z_prime):
            return 1.0 / np.sqrt(O_m * (1 + z_prime)**3 + O_DE)

        # Create redshift sample array
        z_array = np.linspace(0, z, num=1000)

        # Evaluate integrand on array
        y_values = integrand(z_array)

        # Compute definite integral from 0 to z
        integral = cumulative_trapezoid(y_values, z_array, initial=0)[-1]

        # Compute luminosity distance
        return (c * (1 + z) / Ho) * integral

    # Handle scalar and array inputs
    if np.isscalar(redshift):
        return luminosity_distance_single(redshift, Ho, O_m, O_DE)
    else:
        return np.array([luminosity_distance_single(z_i, Ho, O_m, O_DE) for z_i in redshift])

def isotropic_equivalent_energy(redshift, fluence, H0=H0_cgs, Omega_m=Om_default, Omega_DE=Ode_default):
    """
    Calculate the isotropic equivalent energy.

    Parameters:
    -----------
    redshift : float or array
        Redshift value(s)
    fluence : float or array
        Observed fluence (should be in erg/cm^2)
    H0 : float
        Hubble constant in cgs units (s^-1)
    Omega_m : float
        Matter density parameter
    Omega_DE : float
        Dark energy density parameter

    Returns:
    --------
    float or array
        Log10 of isotropic equivalent energy
    """
    d_L = luminosity_distance(redshift, H0_cgs, Omega_m, Omega_DE)
    numerator = 4.0 * np.pi * (d_L ** 2) * fluence
    return np.log10(numerator / (1 + redshift))

def measure_distances(a, b, x, y):
    """
    Measures the distance between a point p = (x,y) and a line of best fit y=ax+b.
    :param p: coordinates of the point
    :param a: slope of the fit
    :param b: y-axis cut of the fit
    :return: scalar for the distance
    """
    # for the line point we use x0=0 and thus y0=b
    numerator = abs(a*x -y +b)
    denominator = np.sqrt(1 + (a ** 2))
    return numerator / denominator

def GoF(m,k,x,y):
    """
    Goodness of fit following the chi-squared formula
    :param m: slope
    :param k: y-axis cut
    :param x: x-values
    :param y: y-values
    :return: chi-squared value
    """
    model = m*x + k
    chi_squared = np.sum((y-model)**2)
    return chi_squared

In [ ]:
# Define cosmological parameter grids
Om = np.linspace(0.0,2.0,100)
Ode = np.linspace(0.0,2.0,100)
k = np.linspace(-25.,-20.,25) # from the previous scatter plot, we can infer that the y-axis cut is around -25 and -20
m = np.linspace(0.0,1.0,25) # the slope must be positive

distances = np.zeros([len(Om), len(Ode), len(df['Eiso'])])
medians = np.zeros([len(Om), len(Ode)])

# this 3D array consist of a Om-Ode 2 grid and the depth corresponding to the distance for each data point (75)
for i in range(len(Om)):
    for j in range(len(Ode)):
        try:
            isotropic_model = isotropic_equivalent_energy(z,S,H0_cgs,Om[i],Ode[j])

            # check for NaN or inf values
            if np.any(~np.isfinite(isotropic_model)):
                distances[i,j,:] = np.inf # instead of NaN which creates problems when finding minimum with argmin
                medians[i,j] = np.inf
                print(f'{i} {j}\tOm={Om[i]:.3f}, Ode={Ode[j]:.3f}, isotropic_model contains infinity or NaN')
                continue

            G = np.zeros([len(m), len(k)]) # G[i,j] corresponds to m[i], k[j]
            for mu in range(len(m)):
                for nu in range(len(k)):
                    G[mu,nu] = GoF(m[mu], k[nu], x=isotropic_model, y=Epeak)

            # extract best fit parameters
            best_fit = np.argmin(G)
            m_index, k_index = np.unravel_index(best_fit, G.shape)
            m_fit, k_fit = m[m_index], k[k_index]
            distances[i,j,:] = measure_distances(m_fit, k_fit, isotropic_model, Epeak)

            #average = np.sum(distances[i,j,:])/len(distances[i,j,:])
            medians[i,j] = np.sum(distances[i,j,:])/len(distances[i,j,:])
            #(mu, sigma) = norm.fit(d[i,j,:]) # gaussian of best fit using scipy
            #x_data = np.linspace(np.min(d[i,j,:]), np.max(d[i,j,:]),100)
            #ax[i,j].plot(x_data, gaussian(x_data, mu, sigma), c='r')
            #ax[i,j].vlines(mu, 0, 20, colors='red', linestyles='--')

            print(f'{i} {j}\tOm={Om[i]:.3f}, Ode={Ode[j]:.3f}, m_fit={m_fit:.3f}, k_fit={k_fit:.3f}, median = {medians[i,j]:.3f}')
            fig, ax = plt.subplots(1,2)
            ax[0].scatter(isotropic_model, Epeak, s=1)
            ax[0].plot(isotropic_model, m_fit*isotropic_model + k_fit, c='r', label=f'fit: m_fit={m_fit:.3f}, k_fit={k_fit:.3f}')
            ax[0].set_xlabel(r'$E_\text{iso}$'); ax[0].set_ylabel(r'$E_\text{peak}$'); ax[0].legend()

            ax[1].hist(distances[i,j,:])
            ax[1].vlines(medians[i,j], 0, 14, colors='r', linestyles='dashed', label=f'median: {medians[i,j]:.3f}')
            ax[1].set_xlabel('data-fit distances'); ax[1].legend()

            plt.tight_layout()
            fig.savefig(f'./figures/Om{Om[i]:.3f}_Ode{Ode[j]:.3f}_{i}{j}.png')
            plt.close()

        except Exception as e:
            print(f'Error at Om={Om[i]:.3f}, Ode={Ode[j]:.3f}: {e}')
            distances[i,j,:] = np.inf # mark as invalid

np.save(f'{len(Om)}_{len(m)}_distances.npy', distances)

In [ ]:
# now let's plot the (Om, Ode)mean surface
def create_mask(data, flat=True):
    # Create a proper mask for unphysical regions
    # instead of a simple mask, the proper mask ensures that values aren't marked as NaN or 0, important for the argmin results
    mask = np.zeros_like(medians, dtype=bool)
    for i in range(len(Om)):
        for j in range(len(Ode)):
            if (Om[i]+Ode[j]>1.2 or # approx flat-universe region
                    Om[i]+Ode[j]<0.8):
                mask[i,j]=flat
            if (Ode[j]>Om[i]+1 or
                    #Ode[j] >= Om[i]** (1/2.32) + 1.0 or # approx no big bang area
                    not np.isfinite(medians[i, j])):
                mask[i,j]=True

    return np.ma.masked_where(mask,data)

def add_constraint_lines(plt):
    plt.plot(Om, 0.5*Om, linestyle='--', color='black', alpha=0.7)
    plt.annotate('accelerating', (1.5,0.80), rotation=26.35)
    plt.annotate('decelerating', (1.5,0.67), rotation=26.35)

    plt.plot(Om, Om**(1/2.32)+1, c='r', alpha=0.7)
    plt.annotate('NO BIG BANG', (0.05,1.55), rotation=45, color='red')

    plt.plot(Om, 1-Om, linestyle='--', color='gray', alpha=0.7)
    plt.annotate('open', (0.75,0.1), rotation=-45)
    plt.annotate('closed', (0.8,0.15), rotation=-45)
    return 0


masked_medians = create_mask(medians, flat=False)

Om_index, Ode_index = np.unravel_index(np.argmin(masked_medians), masked_medians.shape)
Om_fit, Ode_fit = Om[Om_index], Ode[Ode_index]

print(f'Best fit: Om={Om_fit:.3f}, Ode={Ode_fit:.3f}')
print(f'Minimum average distance: {masked_medians[Om_index,Ode_index]:.3}')


# Plot
plt.figure(figsize=(8, 6))
im = plt.contourf(Om, Ode, masked_medians.T, cmap='jet', levels=50)
plt.colorbar(im, label='Distances mean')
plt.scatter(Om_fit, Ode_fit, c='red', marker='x', s=100, linewidth=1, label=fr'Best fit: $\Omega_m$={Om_fit:.2f}, $\Omega_{'{DE}'}$={Ode_fit:.2f}')
plt.scatter(0.3, 0.7, c='black', marker='x', s=100, linewidths=1, label=r'Standard Cosmolgy $\Omega_m = 0.3,\ \Omega_{DE} = 0.7$')

add_constraint_lines(plt.gca())
plt.xlabel(r'$\Omega_m$')
plt.ylabel(r'$\Omega_{DE}$')
plt.legend(loc='upper right')
plt.xlim(0.0,2.0)
plt.ylim(0.0,2.0)
plt.tight_layout()